# <center> Speedup Pandas* workflows using Modin* </center>

## Importing and Organizing Data

In this example we will be generating a **synthetic dataset** and **demonstrating stock Pandas operations running with Modin**.

You can use the default kernel <mark>"Python 3 (Intel® oneAPI 2023.0)"</mark> for this notebook.

Let's start by **importing** all the necessary packages and modules

Let's install Modin with the Ray backend. You can find more details [here](https://modin.readthedocs.io/en/stable/getting_started/installation.html)

In [ ]:
%pip install --user --upgrade modin[ray]

Run the below cell to restart the kernel inorder to load the newly installed packages.

In [ ]:
import os
os._exit(00)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

Run the below cell without any modifications

In [ ]:
# ****** Do not change the code in this cell! It verifies that the notebook is being run correctly! ******

def verify_and_print_times(pandas_time, modin_time):
    if modin_time < pandas_time:
        print(f"Modin was {pandas_time / modin_time:.2f}X faster than stock pandas!")
        return
    print(
        f"Oops, stock pandas appears to be {modin_time / pandas_time:.2f}X faster than Modin in this case. "
        "This is unlikely but could happen sometimes on certain machines/environments/datasets. "
        "One of the most probable reasons is the excessive amount of partitions being assigned to a single worker. "
        "You may visit Modin's optimization guide in order to learn more about such cases and how to fix them: "
        "\nhttps://modin.readthedocs.io/en/latest/usage_guide/optimization_notes/index.html\n\n"
        "But first, verify that you're using the latest Modin version, also, try to use different executions, "
        "for basic usage we recommend non-experimental 'PandasOnRay'.\n"
        "Current configuration is:"
    )
    try:
        from modin.utils import get_current_execution

        execution = get_current_execution()
    except ImportError:
        # for modin version < 0.12.0
        try:
            from modin.utils import get_current_backend

            execution = get_current_backend()
        except ImportError:
            # for modin versions < 0.8.1
            execution = (
                "Can't deduce the current execution, your Modin version is too old!"
            )
    print(f"\tExecution: {execution}")
    try:
        import modin.config as cfg

        print(
            f"\tIs experimental: {cfg.IsExperimental.get()}\n"
            f"\tNumber of CPUs to utilize by Modin (check that Modin uses all CPUs on your machine): {cfg.CpuCount.get()}\n"
            f"\tIs in debug mode (debug mode may perform slower): {cfg.IsDebug.get()}"
        )
    except (ImportError, AttributeError):
        # for modin versions < 0.8.2
        print("\tCan't deduce Modin configuration, your Modin version is too old!")
    import modin

    print(f"\tModin version: {modin.__version__}")

def plotter(outputdict):
    fig = plt.figure(figsize = (10, 5)) 
    plt.bar(outputdict.keys(),outputdict.values(),color='blue',width=0.4)
    plt.xlabel("Python Package")
    plt.ylabel("Runtime(seconds)")
    plt.show()

## How to Use Modin

We will also be importing **stock Pandas as pandas** and **Modin as pd to show differentiation**. <br>You can see importing Modin is simple and **does not require any additional steps.**

In [ ]:
import pandas

Below cell imports and initializes Ray. <br>
If you are running this on the oneAPI Devcloud Jupyter interface, set the parameter **'num_cpus=4'**. This will avoid resource crunch during execution

In [ ]:
import ray
ray.shutdown()
ray.init(num_cpus=4, 
         _memory=16000 * 1024 * 1024, 
         object_store_memory=500 * 1024 * 1024,
         _driver_object_store_memory=500 * 1024 * 1024)

In [ ]:
import modin.pandas as pd

We will now **generate a synthetic dataset** using NumPy and save it to a CSV.

In [ ]:
import os
if not os.path.exists('./foo.csv'):
    array=np.random.randint(low=100,high=10000,size=(2**18,2**8))
    #array
    np.savetxt("foo.csv", array, delimiter=",") #how to generate array

Let's load the csv file as a Pandas dataframe and measure the time taken for that operation. Also, display the first five rows.<br>
For **stock pandas, the dataframe is being stored as `pandas_df`** and for **Modin, the same dataframe is being stored as `modin_df`**.<br>
Let's try running the following cell with Pandas first.

In [ ]:
t0 = time.time()
pandas_df = pandas.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
print(f"Pandas read_csv took : {round((time.time()-t0),2)} seconds")
pandas_df.head()

Now let's run the same code, but use **Modin instead of stock Pandas.** Observe the speedup achieved.

In [ ]:
modin_df=pd.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)]) #warmup
t0 = time.time()
modin_df=pd.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
print(f"Modin read_csv took : {round((time.time()-t0),2)} seconds")
modin_df.head()

Let's now **visualize** this speedup from Modin with a plot!

In [ ]:
t0 = time.time()
pandas_df = pandas.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
pandas_time = time.time()- t0

t1 = time.time()
modin_df = pd.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
modin_time = time.time() - t1

print("Pandas Time(seconds):",pandas_time,"\nModin Time(seconds):",modin_time)
verify_and_print_times(pandas_time, modin_time)
outputDict={"Pandas":pandas_time,"Modin":modin_time}
plotter(outputDict)

## Other DataFrame Function Performance Example
We will now show the speedup in performance from Modin compared to stock Pandas with a few common functions.

Like before, **`pandas_df` is for  stock Pandas**, **`modin_df` is for Modin**.

### `df.mean()`

In [ ]:
# Mean
t2 = time.time()
pandas_df.mean(axis=0)
pandas_time=time.time()- t2
print(" stock Pandas wall time for completion in seconds:",pandas_time)

In [ ]:
# Mean
t3 = time.time()
modin_df.mean(axis=0)
modin_time=time.time()- t3
print("Modin wall time for completion in seconds:",modin_time)

In [ ]:
verify_and_print_times(pandas_time, modin_time)

### `df.applymap`

In [ ]:
# Long apply function
t6 = time.time()
print(pandas_df.applymap(lambda x: x + 1))
pandas_time = time.time() - t6
print(" stock Pandas wall time for completion in seconds:",pandas_time)

In [ ]:
# Long apply function
t7 = time.time()
print(modin_df.applymap(lambda x: x + 1))
modin_time = time.time() - t7
print("Modin wall time for completion in seconds:",modin_time)

In [ ]:
verify_and_print_times(pandas_time, modin_time)

### `pd.concat([df, df])`

In [ ]:
# Concat
t8 = time.time()
print(pandas.concat([pandas_df, pandas_df], axis=0))
pandas_time = time.time() - t8
print("stock Pandas wall time for completion in seconds:",pandas_time)

In [ ]:
# Concat
t9 = time.time()
print(pd.concat([modin_df, modin_df], axis=0))
modin_time = time.time() - t9
print("Modin wall time for completion in seconds:",modin_time)

In [ ]:
verify_and_print_times(pandas_time, modin_time)

## Modin Coverage Examples 
The Modin package supports a large variety of Pandas functions. You can find more info on the coverage [here](https://github.com/modin-project/modin#pandas-api-coverage).<br>
Here are some examples:

### Count

In [ ]:
modin_df.count()

### Filter

In [ ]:
modin_df.filter(regex='0$', axis=1)

### iloc

In [ ]:
modin_df.iloc[0]
modin_df.iloc[-1]
modin_df.iloc[:,0]
modin_df.iloc[:,-1]

## Series

In [ ]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

### DataFrame to NumPy Array

In [ ]:
modin_df.to_numpy()

### Series to NumPy Array

In [ ]:
ser = pd.Series(pd.date_range('2000', periods=2, tz="CET"))
ser.to_numpy(dtype=object)
ser.to_numpy(dtype="datetime64[ns]")

### Set Options

In [ ]:
pd.set_option('compute.use_bottleneck', False)
pd.set_option('compute.use_numexpr', False)

### Unique Function for Series

In [ ]:
pd.unique(pd.Series([2, 1, 3, 3]))

In [ ]:
print("[CODE_SAMPLE_COMPLETED_SUCCESFULLY]")

Finally shutdown the Ray workers after the heavylift.

In [ ]:
ray.shutdown()

### Modin Troubleshooting

Facing issues with Modin. Refer the [troubleshoot guide](https://modin.readthedocs.io/en/stable/getting_started/troubleshooting.html) for a possible fix.

<br>

<br>

### Legal Notices and Disclaimers

Intel technologies’ features and benefits depend on system configuration and may require enabled hardware, software or service activation. Performance varies depending on system configuration. No computer system can be absolutely secure. Check with your system manufacturer or retailer or learn more at www.intel.com.<br>
Cost reduction scenarios described including recommendations are intended as examples of how a given Intel-based product, in the specified circumstances and configurations, may affect future costs and provide cost savings. Circumstances will vary. Intel does not guarantee any costs or cost reduction.<br>
This document contains information on products, services and/or processes in development. All information provided here is subject to change without notice. Contact your Intel representative to obtain the latest forecast, schedule, specifications and roadmaps. <br>
Any forecasts of goods and services needed for Intel’s operations are provided for discussion purposes only. Intel will have no liability to make any purchase in connection with forecasts published in this document.<br>
Intel technologies may require enabled hardware, software or service activation.<br>
Software and workloads used in performance tests may have been optimized for performance only on Intel microprocessors.  <br>
Performance tests, are measured using specific computer systems, components, software, operations and functions.  Any change to any of those factors may cause the results to vary.  You should consult other information and performance tests to assist you in fully evaluating your contemplated purchases, including the performance of that product when combined with other products.   For more complete information visit www.intel.com/benchmarks.<br>

|* Other names and brands may be claimed as the property of others. <br>

Your costs and results may vary. <br>
© Intel Corporation.  Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries.  Other names and brands may be claimed as the property of others.<br>
Copyright 2023 Intel Corporation. 
